### Example creating the stiffness for a linear elasticity element using tensors or matrices

In [10]:
using JuAFEM

In [2]:
E = 200e9
ν = 0.3
λ = E*ν / ((1+ν) * (1 - 2ν))
μ = E / (2(1+ν))
δ(i,j) = i == j ? 1.0 : 0.0
g(i,j,k,l) = λ*δ(i,j)*δ(k,l) + μ*(δ(i,k)*δ(j,l) + δ(i,l)*δ(j,k))

# Create a random symmetric material stiffness
C = rand(SymmetricTensor{4, 2})

Ee = [C[1,1,1,1] C[1,1,2,2] C[1,1,1,2];
      C[2,2,1,1] C[2,2,2,2] C[2,2,1,2];
      C[1,2,1,1] C[1,2,2,2] C[1,2,1,2]];

In [3]:
function_space = Lagrange{2, JuAFEM.RefCube, 1}()
quad_rule = QuadratureRule(Dim{2}, JuAFEM.RefCube(), 2)
fe_values = FEValues(Float64, quad_rule, function_space);

x = [0. 1 1 0;
     0 0 1 1]
x_vec = reinterpret(Vec{2, Float64}, x, (4,));

## Stiffness

In [4]:
function ke_element_mat!{T,dim}(Ke, X::Vector{Vec{2, T}}, fe_values::FEValues{dim}, Ee, B, DB, BDB)
    n_basefuncs = n_basefunctions(get_functionspace(fe_values))
    @assert length(X) == n_basefuncs
    
    reinit!(fe_values, X)

    for q_point in 1:length(JuAFEM.points(get_quadrule(fe_values)))
        
        for i in 1:n_basefuncs
            dNdx = shape_gradient(fe_values, q_point, i)[1]
            dNdy = shape_gradient(fe_values, q_point, i)[2]
            B[1, 2*i - 1] = dNdx
            B[2, 2*i - 0] = dNdy
            B[3, 2*i - 0] = dNdx
            B[3, 2*i - 1] = dNdy
        end
        
        A_mul_B!(DB, Ee, B)
        At_mul_B!(BDB, B, DB)
        scale!(BDB, detJdV(fe_values, q_point))
        for p in 1:size(Ke,1)
            for q in 1:size(Ke,2)
                Ke[p, q] += BDB[p, q]
            end
        end
    end
    
    return Ke
end

ke_element_mat! (generic function with 1 method)

In [5]:
function ke_element!{T,dim}(Ke, X::Vector{Vec{2, T}}, fe_values::FEValues{dim}, D)
    n_basefuncs = n_basefunctions(get_functionspace(fe_values))
    @assert length(X) == n_basefuncs
    reinit!(fe_values, X)
    @inbounds for q_point in 1:length(JuAFEM.points(get_quadrule(fe_values)))
        for a in 1:n_basefuncs
            for b in 1:n_basefuncs
                ∇ϕa = shape_gradient(fe_values, q_point, a)
                ∇ϕb = shape_gradient(fe_values, q_point, b)
                # TODO: This would be faster as ∇ϕa ⋅ C ⋅ ∇ϕb
                Ke_e = C2 ⊡ (∇ϕb ⊗ ∇ϕa) * detJdV(fe_values, q_point)
                for d1 in 1:dim, d2 in 1:dim
                    Ke[dim*(a-1) + d1, dim*(b-1) + d2] += Ke_e[d1,d2]
                end
            end
        end
    end
    return Ke
end

ke_element! (generic function with 1 method)

In [6]:
x = [0. 1 1 0;
     0 0 1 1]
x_vec = reinterpret(Vec{2, Float64}, x, (4,))
Ke2 = zeros(8,8)
# Note the transpose
C2 = Tensor{4, 2}((i,j,k,l) -> C[i,l,k,j])
@time ke_element!(Ke2, x_vec, fe_values, C2)

In [7]:
B = zeros(3, 8)
DB = zeros(3,8)
BDB = zeros(8,8)
Ke = zeros(8,8)
fill!(Ke, 0.0)
ke_element_mat!(Ke, x_vec, fe_values, Ee, B, DB, BDB)

8x8 Array{Float64,2}:
  0.605694     0.341619   -0.216991    …  -0.303402    0.00487851   0.0470835
  0.723057     0.518509    0.095869       -0.395473   -0.360256     0.300418 
  0.00999195   0.0257821   0.242757       -0.0639996  -0.0306442    0.226967 
 -0.157015     0.0280719   0.33449        -0.151107   -0.0701033    0.149403 
 -0.393581    -0.303402    0.00487851      0.341619   -0.216991    -0.085301 
 -0.45867     -0.395473   -0.360256    …   0.518509    0.095869    -0.423453 
 -0.222105    -0.0639996  -0.0306442       0.0257821   0.242757    -0.188749 
 -0.107371    -0.151107   -0.0701033       0.0280719   0.33449     -0.0263676

In [9]:
norm(Ke - Ke2) / norm(Ke)

9.149011931207741e-17